In [287]:
import re
from typing import List


def build_py_func_with_empty_body(prompt: str) -> str:
    # def strlen(string: str) -> int: pass
    return prompt[(prompt.index('def ')):(prompt.index(':\n') + 1)] + ' pass'


def extract_py_function_name_from_prompt(prompt: str) -> str:
    func = build_py_func_with_empty_body(prompt)
    prefix = 'def '
    return func[func.index(prefix) + len(prefix):func.index('(')]

In [288]:
from typing import List


def build_java_public_static_func_with_empty_body(prompt: str) -> str:
    # return prompt[(prompt.index('public static ')):(prompt.index(') {') + 3)] + '}'
    # humaneval-x only
    pattern_1 = ') {'
    pattern_2 = '){'
    pattern_3 = ') throws'
    ending_idx = -1
    
    if pattern_1 in prompt:
        ending_idx = prompt.rindex(pattern_1) + len(pattern_1)
        return prompt[(prompt.rindex('public ')):ending_idx] + '}'
    elif pattern_2 in prompt:
        ending_idx = prompt.rindex(pattern_2) + len(pattern_2)
        return prompt[(prompt.rindex('public ')):ending_idx] + '}'
    elif pattern_3 in prompt:
        ending_idx = prompt.rindex(pattern_3)
        return prompt[(prompt.rindex('public ')):ending_idx] + '{ }'
    else:
        raise ValueError('Cannot find the end of the function body')

def extract_java_function_name_from_prompt(prompt: str) -> str:
    func = build_java_public_static_func_with_empty_body(prompt)
    prefix = 'public'
    type_and_func_name = func[func.index(prefix) + len(prefix):func.index('(')]
    return type_and_func_name.split(' ')[-1]

In [289]:
def audit_translated_prompt(self, source_code, target_prompt):
    py_func_name = extract_py_function_name_from_prompt(source_code).replace('_', '').lower()
    java_func_name = extract_java_function_name_from_prompt(target_prompt).lower()
    
    if py_func_name != java_func_name:
        raise ValueError(f"Formalized function names of source and target prompts do not match: {py_func_name} <> {java_func_name}")

In [290]:
source_path = '/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/CodeLlama-13b-Python-hf/py/improve/pass@1/t0.2-p0.95-k0-batch1-n1/CodeLlama-13b-Python-hf-temp0.2-p0.95-k0-bf16-n1-seed5-batch1-maxlen1024-py-generations-0-158_multiple-py.json'

# tgt_path = '/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_java.json'

# tgt_path = '/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/MultiPL-E/humaneval-java-reworded.json'

tgt_path = '/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/bigcode_eval/tasks/code_to_code/datasets/humaneval-py-java-refined.json'

In [291]:
import json
source_gens = json.load(open(source_path, 'r'))
tgt_tasks = json.load(open(tgt_path, 'r'))

print(f'tgt_tasks={len(tgt_tasks)}')

# build target {task -> prompt}
tgt_map = {}

for tgt_task in tgt_tasks:
    tgt_prompt = tgt_task['prompt']
    try:
        java_func_name = extract_java_function_name_from_prompt(tgt_prompt).lower()
        
        if java_func_name in tgt_map:
            print(f'existing:{tgt_map[java_func_name]}')
            print(f'current:{tgt_prompt}')
            raise Exception(f'java_func_name={java_func_name} already exists')
    
        
        tgt_map[java_func_name] = tgt_task
    except Exception as ex:
        print(ex)
        print(tgt_prompt)
        
print(f'java_func_names={tgt_map.keys()}')

for task_gens in source_gens:
    task_gen = task_gens[0]
    task_py_func_name = extract_py_function_name_from_prompt(
        task_gen).lower().replace('_', '')

    tgt_prompt = tgt_map[task_py_func_name]

    assert tgt_prompt

    # print(task_py_func_name)

tgt_tasks=158
java_func_names=dict_keys(['strlen', 'encrypt', 'checkdictcase', 'add', 'fixspaces', 'fibfib', 'doublethedifference', 'filterintegers', 'carracecollision', 'parsemusic', 'decimaltobinary', 'allprefixes', 'addtwonumbers', 'eat', 'maxfill', 'doalgebra', 'flipcase', 'bylength', 'factorize', 'countupto', 'unique', 'totalmatch', 'maxelement', 'isnested', 'roundedavg', 'oddcount', 'moveoneball', 'evenoddpalindrome', 'isequaltosumeven', 'derivative', 'issorted', 'solve', 'tri', 'fizzbuzz', 'filterbyprefix', 'solvesumdigitsinbin', 'minpath', 'countupper', 'maximum', 'largestdivisor', 'sortarray', 'f', 'iscube', 'encode', 'isbored', 'pairssumtozero', 'trianglearearounded', 'bf', 'digits', 'wordsstring', 'howmanytimes', 'compareone', 'removevowels', 'strangesortlist', 'findclosestelements', 'issimplepower', 'primefib', 'orderbypoints', 'hascloseelements', 'makepalindrome', 'stringxor', 'specialfactorial', 'addelements', 'fib4', 'uniquedigits', 'selectwords', 'willitfly', 'fib', 'st

In [292]:

src_map = {}

for src_gens in source_gens:
    src_prompt = src_gens[0]
    try:
        py_func_name = extract_py_function_name_from_prompt(
            src_prompt).lower().replace('_', '')

        if py_func_name in src_map:
            raise Exception(f'py_func_name={py_func_name} already exists')

        src_map[py_func_name] = tgt_task
    except Exception as ex:
        print(ex)
        print(src_prompt)
        raise ex

print(f'source_gens={len(source_gens)}')
print(f'py_func_names={src_map.keys()}')

for tgt_task in tgt_tasks:
    tgt_prompt = tgt_task['prompt']

    tgt_java_func_name = extract_java_function_name_from_prompt(
        tgt_prompt).lower().replace('_', '')

    assert tgt_java_func_name
    assert src_map[tgt_java_func_name] is not None

source_gens=158
py_func_names=dict_keys(['strlen', 'encrypt', 'checkdictcase', 'add', 'fixspaces', 'fibfib', 'doublethedifference', 'filterintegers', 'carracecollision', 'parsemusic', 'decimaltobinary', 'allprefixes', 'addtwonumbers', 'eat', 'maxfill', 'doalgebra', 'flipcase', 'bylength', 'factorize', 'countupto', 'unique', 'totalmatch', 'maxelement', 'isnested', 'roundedavg', 'oddcount', 'moveoneball', 'evenoddpalindrome', 'isequaltosumeven', 'derivative', 'issorted', 'solve', 'tri', 'fizzbuzz', 'filterbyprefix', 'solvesumdigitsinbin', 'minpath', 'countupper', 'maximum', 'largestdivisor', 'sortarray', 'f', 'iscube', 'encode', 'isbored', 'pairssumtozero', 'trianglearearounded', 'bf', 'digits', 'wordsstring', 'howmanytimes', 'compareone', 'removevowels', 'strangesortlist', 'findclosestelements', 'issimplepower', 'primefib', 'orderbypoints', 'hascloseelements', 'makepalindrome', 'stringxor', 'specialfactorial', 'addelements', 'fib4', 'uniquedigits', 'selectwords', 'willitfly', 'fib', 'st

In [293]:
multiple_java_path = '/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/MultiPL-E/humaneval-java-reworded.json'